# Equations of motion for a "zagi-like" flying wing UAV

Do imports.

In [1]:
import numpy as np
import sympy as sym
from scipy import linalg
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

Define variables and parameters as symbols.

In [2]:
#
# VARIABLES
#

# Time
t = sym.Symbol('t', real=True)

# Components of position (meters)
p_x, p_y, p_z = sym.symbols('p_x, p_y, p_z', real=True)

# Yaw, pitch, and roll angles (radians)
psi, theta, phi = sym.symbols('psi, theta, phi', real=True)

# Components of linear velocity in the body frame (meters / second)
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z', real=True)

# Components of angular velocity in the body frame (radians / second)
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z', real=True)

# Elevon angles
delta_r, delta_l = sym.symbols('delta_r, delta_l', real=True)

#
# PARAMETERS
#

# Aerodynamic parameters
rho, S, c, b = sym.symbols('rho, S, c, b', real=True)
C_L_0, C_L_alpha, C_L_q, C_L_delta_e = sym.symbols('C_L_0, C_L_alpha, C_L_q, C_L_delta_e', real=True)
C_D_0, C_D_alpha, C_D_q, C_D_delta_e = sym.symbols('C_D_0, C_D_alpha, C_D_q, C_D_delta_e', real=True)
C_m_0, C_m_alpha, C_m_q, C_m_delta_e = sym.symbols('C_m_0, C_m_alpha, C_m_q, C_m_delta_e', real=True)
C_Y_0, C_Y_beta, C_Y_p, C_Y_r, C_Y_delta_a = sym.symbols('C_Y_0, C_Y_beta, C_Y_p, C_Y_r, C_Y_delta_a', real=True)
C_l_0, C_l_beta, C_l_p, C_l_r, C_l_delta_a = sym.symbols('C_l_0, C_l_beta, C_l_p, C_l_r, C_l_delta_a', real=True)
C_n_0, C_n_beta, C_n_p, C_n_r, C_n_delta_a = sym.symbols('C_n_0, C_n_beta, C_n_p, C_n_r, C_n_delta_a', real=True)
e, alpha_0, C_D_p, M = sym.symbols('e, alpha_0, C_D_p, M', real=True)
k, k_e = sym.symbols('k, k_e', real=True)

# Mass and inertia parameters
J_x, J_y, J_z, J_xz = sym.symbols('J_x, J_y, J_z, J_xz', real=True)
m, g = sym.symbols('m, g', real=True)

Define numerical values of parameters.

In [3]:
params = {
    g: 9.81,               # Gravity (m/s²)
    m: 1.56,               # Mass of the UAV (kg)
    J_x: 0.1147,           # Moment of inertia about x-axis (kg·m²) [UPDATED 02/28/2025]
    J_y: 0.0576,           # Moment of inertia about y-axis (kg·m²) [UPDATED 02/28/2025]
    J_z: 0.1712,           # Moment of inertia about z-axis (kg·m²) [UPDATED 02/28/2025]
    J_xz: 0.0015,          # Product of inertia (kg·m²)             [UPDATED 02/28/2025]

    S: 0.4696,             # Wing area (m²)
    b: 1.4224,             # Wingspan (m)
    c: 0.3302,             # Mean aerodynamic chord (m)

    rho: 1.2682,           # Air density (kg/m³)

    # Lift Coefficients
    C_L_0: 0.2,            # Lift coefficient at zero AoA
    C_L_alpha: 4.8,        # Lift curve slope (1/rad)
    C_L_q: 2.2,            # Pitch rate effect on lift (1/rad)

    # Drag Coefficients
    C_D_0: 0.02,           # Zero-lift drag coefficient
    C_D_alpha: 0.30,       # Drag change per AoA (1/rad)
    C_D_q: 0.0,            # Pitch rate effect on drag (1/rad)
    C_D_p: 0.03,           # Parasitic drag coefficient

    # Pitching Moment Coefficients
    C_m_0: -0.02,          # Pitching moment at zero AoA
    C_m_alpha: -0.6,       # Pitching moment change per AoA (1/rad)
    C_m_q: -1.8,           # Pitch rate effect on moment (1/rad)
    C_m_delta_e: -0.35,    # Effect of elevator deflection on pitching moment (1/rad)

    # Side Force Coefficients
    C_Y_0: 0.0,            # Side force at zero sideslip
    C_Y_beta: -0.08,       # Side force per sideslip angle (1/rad)
    C_Y_p: 0.0,            # Side force due to roll rate
    C_Y_r: 0.0,            # Side force due to yaw rate
    C_Y_delta_a: 0.0,      # Side force due to aileron deflection

    # Roll Moment Coefficients
    C_l_0: 0.0,            # Roll moment at zero sideslip
    C_l_beta: -0.10,       # Roll moment due to sideslip (1/rad)
    C_l_p: -0.45,          # Roll damping derivative (1/rad)
    C_l_r: 0.03,           # Roll moment due to yaw rate (1/rad)
    C_l_delta_a: 0.18,     # Aileron effect on roll (1/rad)

    # Yaw Moment Coefficients
    C_n_0: 0.0,            # Yaw moment at zero sideslip
    C_n_beta: 0.008,       # Yaw moment due to sideslip (1/rad)
    C_n_p: -0.022,         # Yaw moment due to roll rate (1/rad)
    C_n_r: -0.009,         # Yaw damping derivative (1/rad)
    C_n_delta_a: -0.004,   # Aileron effect on yaw (1/rad)

    # Control Derivatives
    C_L_delta_e: 0.30,     # Effect of elevator deflection on lift (1/rad)
    C_D_delta_e: 0.32,     # Effect of elevator deflection on drag (1/rad)

    # Efficiency Factors
    e: 0.85,               # Oswald efficiency factor
    alpha_0: 0.45,         # Zero-lift angle of attack (rad)

    # Additional Drag & Lift Coefficients
    M: 50.0,               # Sigmoid blending function parameter
    k_e: 0.01,             # Drag due to elevator deflection (empirical coefficient)
    k: 0.048               # Induced drag factor
}

Compute aerodynamic forces and torques (assuming low angles of attack, so no stall).

In [4]:
# Get airspeed, angle of attack, and angle of sideslip
V_a = sym.sqrt(v_x**2 + v_y**2 + v_z**2)
alpha = sym.atan(v_z / v_x)
beta = sym.asin(v_y / V_a)

# Convert from right and left elevon deflections to equivalent elevator and aileron deflections
delta_e = (delta_r + delta_l) / 2
delta_a = (-delta_r + delta_l) / 2

# Longitudinal aerodynamics
C_L = C_L_0 + C_L_alpha * alpha
F_lift = rho * V_a**2 * S * (C_L + C_L_q * (c / (2 * V_a)) * w_y + C_L_delta_e * delta_e) / 2
F_drag = rho * V_a**2 * S * ((C_D_0 + k * C_L**2) + C_D_q * (c / (2 * V_a)) * w_y + k_e * (C_L_delta_e * delta_e)**2) / 2
f_x, f_z = sym.Matrix([[sym.cos(alpha), -sym.sin(alpha)], [sym.sin(alpha), sym.cos(alpha)]]) @ sym.Matrix([[-F_drag], [-F_lift]])
tau_y = rho * V_a**2 * S * c * (C_m_0 + C_m_alpha * alpha + C_m_q * (c / (2 * V_a)) * w_y + C_m_delta_e * delta_e) / 2

# Lateral aerodynamics
f_y =   rho * V_a**2 * S *     (C_Y_0 + C_Y_beta * beta + C_Y_p * (b / (2 * V_a)) * w_x + C_Y_r * (b / (2 * V_a)) * w_z + C_Y_delta_a * delta_a) / 2
tau_x = rho * V_a**2 * S * b * (C_l_0 + C_l_beta * beta + C_l_p * (b / (2 * V_a)) * w_x + C_l_r * (b / (2 * V_a)) * w_z + C_l_delta_a * delta_a) / 2
tau_z = rho * V_a**2 * S * b * (C_n_0 + C_n_beta * beta + C_n_p * (b / (2 * V_a)) * w_x + C_n_r * (b / (2 * V_a)) * w_z + C_n_delta_a * delta_a) / 2

Define the linear velocity vector $v^B_{W, B}$ and the angular velocity vector $w^B_{W, B}$ in the coordinates of the body frame.

In [5]:
v_inB_ofWB = sym.Matrix([v_x, v_y, v_z])
w_inB_ofWB = sym.Matrix([w_x, w_y, w_z])

Define the moment of inertia matrix in the coordinates of the body frame.

In [6]:
J_inB = sym.Matrix([[  J_x,    0, -J_xz],
                    [    0,  J_y,     0],
                    [-J_xz,    0,   J_z]])

Define individual rotation matrices.

In [7]:
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                 [sym.sin(psi), sym.cos(psi), 0],
                 [0, 0, 1]])

Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)],
                 [0, 1, 0],
                 [-sym.sin(theta), 0, sym.cos(theta)]])

Rx = sym.Matrix([[1, 0, 0],
                 [0, sym.cos(phi), -sym.sin(phi)],
                 [0, sym.sin(phi), sym.cos(phi)]])

Apply sequential transformation to compute the rotation matrix that describes the orientation of the aircraft (i.e., of frame $B$ in the coordinates of frame $W$).

In [8]:
R_inW_ofB = Rz @ Ry @ Rx

Compute the matrix $N$ for which

$$\begin{bmatrix} \dot{\psi} \\ \dot{\theta} \\ \dot{\phi} \end{bmatrix} = N w_{W, B}^{B}.$$

In [9]:
# First, compute the inverse of N
Ninv = sym.Matrix.hstack((Ry * Rx).T * sym.Matrix([0, 0, 1]),
                              (Rx).T * sym.Matrix([0, 1, 0]),
                                       sym.Matrix([1, 0, 0]))

# Then, take the inverse of this result to compute N
N = sym.simplify(Ninv.inv())

Compute the total force and torque on the aicraft in the body frame (remember that $z$ is down!).

In [10]:
# Total force
f_inB = R_inW_ofB.T * sym.Matrix([0, 0, m * g]) + sym.Matrix([f_x, f_y, f_z])

# Total torque
tau_inB = sym.Matrix([tau_x, tau_y, tau_z])

Define (fully) symbolic equations of motion. These are complicated enough that we won't bother to show them.

In [11]:
f_sym = sym.Matrix.vstack(
    R_inW_ofB * v_inB_ofWB,
    N * w_inB_ofWB,
    (1 / m) * (f_inB - w_inB_ofWB.cross(m * v_inB_ofWB)),
    J_inB.inv() * (tau_inB - w_inB_ofWB.cross(J_inB * w_inB_ofWB)),
)

Substitute numerical values for each parameter. The only symbols that remain in these equations of motion are the (nonlinear) states and inputs. Again, these equations of motion are complicated, so we won't bother to show them. They have the following form:

$$
\begin{bmatrix} \dot{p}_x \\ \dot{p}_y \\ \dot{p}_z \\ \dot{\psi} \\ \dot{\theta} \\ \dot{\phi} \\ \dot{v}_x \\ \dot{v}_y \\ \dot{v}_z \\ \dot{w}_x \\ \dot{w}_y \\ \dot{w}_z \end{bmatrix}
=
f\left(
\begin{bmatrix} p_x \\ p_y \\ p_z \\ \psi \\ \theta \\ \phi \\ v_x \\ v_y \\ v_z \\ w_x \\ w_y \\ w_z \end{bmatrix},
\begin{bmatrix} \delta_r \\ \delta_l \end{bmatrix}
\right).
$$

In [12]:
f = f_sym.subs(params)

In [13]:
# Substitute the numerical parameters into the equations
f_uav = sym.simplify(f_sym.subs(params))

f_uav_reduced = f_uav[[1, 3, 4,5,6,7,8,9,10,11], :]

f_uav_num_reduced = sym.lambdify(
    [p_y, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, delta_r, delta_l],
    f_uav_reduced
)

from scipy.optimize import minimize

f_num_for_minimize = lambda x: np.linalg.norm(f_uav_num_reduced(*x).flatten())**2

# Initial guess
x0 = [
    0., # <-- guess at p_y_e
    0., # <-- guess at psi_e
    0., # <-- guess at theta_e
    0., # <-- guess at phi_e
    5., # <-- guess at v_x_e - ESTIMATED MANUALLY BY PERFORMING NUMEROUS ITERATIONS
    0., # <-- guess at v_y_e
    0.15, # <-- guess at v_z_e - ESTIMATED MANUALLY BY PERFORMING NUMEROUS ITERATIONS
    0., # <-- guess at w_x_e
    0., # <-- guess at w_y_e
    0., # <-- guess at w_z_e
    0., # <-- guess at delta_r_e
    0., # <-- guess at delta_l_e

]

# Find minimum ("tol" is a tolerance that says how accurate you want the solution)
sol = minimize(f_num_for_minimize, x0, tol=1e-4)

# Show solution
print(sol)

# Extract equilibrium state variables from the solution vector
(p_y_e, psi_e, theta_e, phi_e, v_x_e, v_y_e, v_z_e, w_x_e, w_y_e, w_z_e, delta_r_e, delta_l_e) = sol.x

# Print the equilibrium state values with appropriate formatting:
print(f'  p_y_e   = {p_y_e:7.3f}')
print(f'  psi_e   = {psi_e:7.3f}')
print(f'  theta_e = {theta_e:7.3f}')
print(f'  phi_e   = {phi_e:7.3f}')
print(f'  v_x_e   = {v_x_e:7.3f}')
print(f'  v_y_e   = {v_y_e:7.3f}')
print(f'  v_z_e   = {v_z_e:7.3f}')
print(f'  w_x_e   = {w_x_e:7.3f}')
print(f'  w_y_e   = {w_y_e:7.3f}')
print(f'  w_z_e   = {w_z_e:7.3f}')
print(f'  delta_r_e   = {delta_r_e:7.3f}')
print(f'  delta_l_e   = {delta_l_e:7.3f}')

# Find A and B in symbolic form
A_sym = f_uav_reduced.jacobian([p_y, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z])
B_sym = f_uav_reduced.jacobian([delta_r, delta_l])

# Create lambda functions to allow numerical evaluation of A and B
A_num = sym.lambdify([p_y, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, delta_r, delta_l], A_sym)
B_num = sym.lambdify([p_y, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, delta_r, delta_l], B_sym)

# Find A and B in numeric form (making sure the result is floating-point)
A = A_num(p_y_e, psi_e, theta_e, phi_e, v_x_e, v_y_e, v_z_e, w_x_e, w_y_e, w_z_e, delta_r_e, delta_l_e).astype(float)
B = B_num(p_y_e, psi_e, theta_e, phi_e, v_x_e, v_y_e, v_z_e, w_x_e, w_y_e, w_z_e, delta_r_e, delta_l_e).astype(float)

# Show state-space model
print(f'A =\n{A}\n\nB =\n{B}')

def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @  B.T @ P
    return K, P

# Region 1: Low error (|ψ|,|θ|,|φ| ≤ π/18 ≈ 10°)
Q1 = np.diag([1., 0.5, 0.5, 0.5, 1., 1., 1., 1., 1., 1.])
R1 = np.diag([1.0, 1.0])
# Region 2: Moderate error (π/18 < error ≤ 0.40 rad ≈ 23°)
Q2 = np.diag([1.0, 0.10, 0.10, 0.10, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
R2 = np.diag([2.0, 2.0])
# Region 3: High error (0.40 < error ≤ π/6 ≈ 0.5236 rad)
Q3 = np.diag([1.0, 0.01, 0.01, 0.01, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
R3 = np.diag([5.0, 5.0])

# Compute gain matrices for each region (A and B are assumed to be computed previously)
K1, P1 = lqr(A, B, Q1, R1)
K2, P2 = lqr(A, B, Q2, R2)
K3, P3 = lqr(A, B, Q3, R3)

print("K1 =", K1.tolist())
print(f'p = {linalg.eigvals(A - B @ K1)}')
print("K2 =", K2.tolist())
print(f'p = {linalg.eigvals(A - B @ K2)}')
print("K3 =", K3.tolist())
print(f'p = {linalg.eigvals(A - B @ K3)}')

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 2.2596702921779166e-11
        x: [ 0.000e+00 -1.056e-05  2.225e-01 -1.253e-07  5.345e+00
             5.881e-05  1.835e+00  1.279e-06  8.030e-07 -6.925e-07
            -6.239e-01 -6.239e-01]
      nit: 30
      jac: [ 0.000e+00 -2.789e-06  5.183e-06 -1.861e-06 -3.150e-06
             1.828e-06  2.317e-06  4.392e-06  9.737e-06  8.554e-07
             2.157e-05  5.523e-06]
 hess_inv: [[ 1.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
            [ 0.000e+00  2.116e+00 ...  1.268e+00 -1.263e+00]
            ...
            [ 0.000e+00  1.268e+00 ...  9.536e-01 -6.465e-01]
            [ 0.000e+00 -1.263e+00 ... -6.465e-01  9.429e-01]]
     nfev: 468
     njev: 36
  p_y_e   =   0.000
  psi_e   =  -0.000
  theta_e =   0.223
  phi_e   =  -0.000
  v_x_e   =   5.345
  v_y_e   =   0.000
  v_z_e   =   1.835
  w_x_e   =   0.000
  w_y_e   =   0.000
  w_z_e   =  -0.000
  delta_r_e   =  -0.624
  delta_l_e   =  -